In [ ]:
import tkinter as tk
import requests
import random
from bs4 import BeautifulSoup
from PIL import Image, ImageTk
from io import BytesIO

# Transformando tk_img numa variável global
tk_img = None

# Gerando 8-bits randomicos
def create_cool_graphic():
   
    window = tk.Tk()
    window.title("8-Bit Aleatório")
    window.geometry("300x300")

    # Criando canvas
    canvas = tk.Canvas(window, width=300, height=300, bg="white")
    canvas.pack()

    # Randomm
    for x in range(8):
        for y in range(8):
            color = random.randint(0, 1) * 255
            canvas.create_rectangle(x * 30, y * 30, (x + 1) * 30, (y + 1) * 30, fill=f"#{color:02x}{color:02x}{color:02x}")

    # Loop
    window.mainloop()


# Criando notepad no Tkinter
def create_notepad():
    global tk_img
    

    # Criando janela do notepad
    notepad_window = tk.Toplevel()
    notepad_window.geometry("400x400")
    notepad_window.title("Notepad")
 

    # Criando texto e adicionando na janela principal
    notepad_text = tk.Text(notepad_window)
    notepad_text.pack(fill=tk.BOTH, expand=True)

    # Adicionando scrollbar
    scrollbar = tk.Scrollbar(notepad_text)
    scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
    notepad_text.config(yscrollcommand=scrollbar.set)
    scrollbar.config(command=notepad_text.yview)

# Web scrapping para obter informações e fotos diárias do NASA APOD usando Beautiful Soup

def get_image_info2():
    global tk_img
    
    # URL da página da NASA APOD 
    url = "https://apod.nasa.gov/apod/"

    # Fazendo request pro HTML
    response = requests.get(url)
    html_content = response.content

    # Web Scrapping com Beautiful Soup
    soup = BeautifulSoup(html_content, 'html.parser')

    # Extraindo o título da foto do HTML
    title = soup.b.text

    # Achando a tag <img> que tem a imagem e inserindo a url em outra variável
    img_tag = soup.find('img')
    img_url = f"{url}{img_tag['src']}"

    # Carregando imagem da URL usando pillow
    response = requests.get(img_url)
    img = Image.open(BytesIO(response.content))

    # Ajustando tamanho da imagem pra caber na janela
    img = img.resize((700, 400), Image.ANTIALIAS)

    # Convertendo a imagem para uma formato suportado no Tkinter
    tk_img = ImageTk.PhotoImage(img)

    # Criando uma nova janela pra imagem e informação da NASA
    new_window = tk.Toplevel()
    new_window.title("Image Info")
    new_window.geometry("800x600")

    # Criando um label para exibir a imagem
    img_label = tk.Label(new_window, image=tk_img)
    img_label.pack()

    # Iterando os <p> para achar o texto de explicação
    p_tags = soup.find_all('p')
    explanation = None
    for p in p_tags:
        if 'Explanation:' in p.get_text():
            explanation = p.get_text()
            break

    # Exibindo título e explicação no Tkinter
    text = f"{title}\n\n{explanation}"
    explanation_text = tk.Text(new_window, wrap=tk.WORD, font=('Arial', 12))
    explanation_text.insert(tk.END, text)
    explanation_text.pack(padx=10, pady=10)

# Criando janela Tkinter 
root = tk.Tk()
root.configure(bg="black")
root.title("Gerador NASA APOD")

# Setando o tamanho
root.geometry("400x200")

# Centralizando janela na tela
screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()
x_coordinate = int((screen_width/2) - (800/2))
y_coordinate = int((screen_height/2) - (600/2))
root.geometry(f"+{x_coordinate}+{y_coordinate}")



# Crianto botoões 
white_button = tk.Button(root, text="☁ 8-BIT ☁", bg="grey", fg="blue", command=create_cool_graphic)
white_button.pack(pady=10)
white_button = tk.Button(root, text="★ GERAR IMAGEM DA NASA ★", bg="white", fg="blue", command=get_image_info2)
white_button.pack(pady=20)
black_button = tk.Button(root, text="☁ ABRIR BLOCO DE NOTAS ☁", bg="pink", fg="blue", command=create_notepad)
black_button.pack(pady=10)

# Start the Tkinter mainloop
root.mainloop()
